In [5]:
from helper_fn import *
import re
import spacy
import scispacy
from spacy import displacy
from spacy.tokenizer import Tokenizer
# For LDA 
from gensim.utils import simple_preprocess
from gensim.models import LdaMulticore
import gensim.corpora as corpora
# For word clouds
from wordcloud import WordCloud
# For most frequent words
from collections import Counter

# TODO LIST (April 17st, 2022) 

1. Most common word counts using spacy. Biomedical terms recognition.
2. LDA on larger bag of words. To ask: How did you decide on those 600 videos? 
3. Identify words that have highest weight for each video. Pick most common topics between the two.
Top 10 most common words of understandable videos contain more diabetes-related terms than those that don't. However,
I haven't normalized the word per duration yet due to storing issue. E.g., genesthey&#39;re
4. Descriptive statisitcs for each class (Done)
5. LASSO sparse regression on word counts and other variables Xiao used.

6. Can embed the similarity between words
7. Sentiment analysis

Summary on misinformation
- 469 (3/4) of the videos are labelled, of which 30% are not misinformation and the rest are. 

In [6]:
# Download videos with subtitles and dictionary of videos (key = ID, labels = subtitle)
df600 = pd.read_csv("merged_and_cleaned600.csv", sep = ",").drop(columns = ["Unnamed: 0"])
outdir = "/Users/lerdp/Desktop"
l_file = open("%s/labelled_subtitle.pkl" %(outdir),'rb')
ul_file = open("%s/unlabelled_subtitle.pkl" %(outdir), 'rb')
l_subtitle = pickle.load(l_file)
ul_subtitle = pickle.load(ul_file)
l_file.close()
ul_file.close()

# Define nlp spacy
nlp_medical = spacy.load("en_core_sci_md")
nlp = spacy.load("en_core_web_sm")

In [79]:
def getID(URL):
    if URL == None:
        return
    else:
        URL = re.sub(r'http\S+=', '', URL)
        URL = re.sub(r'http\S+/', '', URL)
        return URL

def isMisinfo(text):
    if text.lower().strip() == "yes":
        return 1
    else:
        return 0

def getWordBag(doc):
    allwords = [token.lemma_ for token in doc if not token.is_punct and not token.is_stop 
                                              and not token.is_space and token.is_alpha]
    medwords = [ent.lemma_.lower() for ent in doc.ents]
    return allwords, medwords

def wordFrequency(word_lst, ntop = 5, normalize = False):
    temp = {}
    total_word = 0
    for word in word_lst:
        if word.strip() != '': # Not an empty string
            try:
                temp[word] += 1
            except:
                temp[word] = 1
            total_word += 1
    if normalize:
        for word in list(temp.keys()):
            temp[word] = round(temp[word]/total_word, 4)
    return dict(Counter(temp).most_common(ntop))

In [10]:
# Download misinformation labels and rename columns
df_mis = pd.read_csv("misinformation.csv", lineterminator = "\r")
df_mis = df_mis.drop(columns = df_mis.columns[4:]).dropna(subset = ["URL"])
df_mis["video_id"] = df_mis["URL"].map(getID)

# Rename the columns
rename = {}
for col in df_mis.columns.tolist():
    rename[col] = col.lower().strip()
df_mis = df_mis.rename(columns=rename)

# Obtain the video_id
cur_index = df_mis[df_mis["title"] == "Latent autoimmune diabetes of adults"].index.tolist()[0]
df_mis = df_mis[:cur_index]

# Convert misinformation into indicator
df_mis["misinformation"] = df_mis["misinformation"].fillna("No")
df_mis["misinformation"] = df_mis["misinformation"].map(isMisinfo)

In [40]:
# TODO: Make more elegant + remove items I don't need.
# Idea:  
mis_allfreqlst = [] # List of
mis_medfreqlst = [] # List of frequencies for each word
cor_allfreqlst = [] # List of all words frequency for each of the videos 
cor_medfreqlst = [] # List of medical word frequency for correct videos
emptysub = [] # List of IDs for videos with empty subtitles
ntop = 15

cor_allwords_lst = []
mis_allwords_lst = []

all_id = list(l_subtitle.keys()) + list(ul_subtitle.keys())
mis_nnosub = 0
cor_nnosub = 0
mis_alltext = ""
cor_alltext = ""

# Correct videos: Create a list of subtitles for each video; two dictionaries for all and medical frequency
for video_id in df_mis[df_mis["misinformation"] == 0]["video_id"]:
    if video_id in all_id:
        try:
            subtitle = l_subtitle[video_id]
        except:
            subtitle = ul_subtitle[video_id]
        if subtitle.strip() != "":
            # Modify the line below for processing
            doc_medical = nlp_medical(subtitle)
            allwords, medwords = getWordBag(doc_medical)
            cor_allfreqlst.append(wordFrequency(allwords, ntop = ntop, normalize = True))
            cor_medfreqlst.append(wordFrequency(medwords, ntop = ntop, normalize = True))
            cor_alltext += subtitle
            cor_allwords_lst.append(allwords)
        else: 
            emptysub.append(video_id)
    else:
        cor_nnosub += 1
        
# Misinformation videos: Create a list of subtitles for each video; two dictionaries for all and medical frequency
for video_id in df_mis[df_mis["misinformation"] == 1]["video_id"]:
    if video_id in all_id:
        try:
            subtitle = l_subtitle[video_id]
        except:
            subtitle = ul_subtitle[video_id]
        if subtitle.strip() != "":
            # Modify the line below for processing
            doc_medical = nlp_medical(subtitle)
            allwords, medwords = getWordBag(doc_medical)
            mis_allfreqlst.append(wordFrequency(allwords, ntop = ntop, normalize = True))
            mis_medfreqlst.append(wordFrequency(medwords, ntop = ntop, normalize = True))
            mis_alltext += subtitle
            mis_allwords_lst.append(allwords)
        else:  # Exclude all videos with empty subtitles
            emptysub.append(video_id)
    else:
        mis_nnosub += 1

In [12]:

doc_medical_misinfo = nlp_medical(mis_alltext)
doc_medical_cor = nlp_medical(cor_alltext)

In [30]:
allwords

['latent',
 'autoimmune',
 'diabete',
 'adult',
 'late',
 'onset',
 'autoimmune',
 'diabete',
 'adulthood',
 'aging',
 'slow',
 'onset',
 'type',
 'diabete',
 'diabetes',
 'type',
 'form',
 'diabetes',
 'mellitus',
 'type',
 'occur',
 'adult',
 'slow',
 'course',
 'onset',
 'adult',
 'la',
 'da',
 'initially',
 'diagnose',
 'have',
 'type',
 'diabete',
 'base',
 'age',
 'particularly',
 'risk',
 'factor',
 'type',
 'diabete',
 'strong',
 'family',
 'history',
 'obesity',
 't',
 'diagnosis',
 'base',
 'finding',
 'high',
 'blood',
 'sugar',
 'clinical',
 'impression',
 'islet',
 'failure',
 'insulin',
 'resistance',
 'main',
 'cause',
 'detection',
 'low',
 'raise',
 'antibody',
 'islet',
 'langerhan',
 'support',
 'diagnosis',
 'treat',
 'usual',
 'oral',
 'treatment',
 'type',
 'diabete',
 'certain',
 'period',
 'time',
 'insulin',
 'treatment',
 'usually',
 'necessary',
 'monitoring',
 'complication',
 'l',
 'Ada',
 'introduce',
 'sign',
 'symptom',
 'symptom',
 'latent',
 'autoimmun

In [130]:
temp1 = ""
comment_lst = df_mis["comment"].value_counts().index
comment = comment_lst[4]
print(comment)
for video_id in df_mis[df_mis["comment"] == comment]["video_id"]:
    if video_id in all_id:
        try:
            subtitle = l_subtitle[video_id]
        except:
            subtitle = ul_subtitle[video_id]
        temp1 += subtitle
doc_medical = nlp_medical(temp1)
doc = nlp(temp1)

awtemp, mwtemp = getWordBag(doc_medical)
print("Ratio of medical words to allwords = %.2f" %(len(mwtemp)/len(awtemp)))
wordFrequency(mwtemp, ntop = 10)

Sales pitch at beginning of video is annoying
Ratio of medical words to allwords = 0.52


{'diabete': 123,
 'pre-diabete': 42,
 'meal': 39,
 'blood sugar': 37,
 'type 2 diabete': 34,
 'people': 29,
 'food': 27,
 'a1c': 26,
 'i': 25,
 'video': 24}

In [20]:
allwords_cor, medwords_cor = getWordBag(doc_medical_cor)
wordFrequency(medwords_cor, ntop = 10, normalize=True)

{'diabete': 0.0247,
 'insulin': 0.0209,
 'blood sugar': 0.0103,
 'people': 0.0074,
 'level': 0.0067,
 'meal': 0.0065,
 'sugar': 0.0062,
 'patient': 0.006,
 'type 2 diabete': 0.0057,
 'day': 0.0057}

In [21]:
allwords_misinfo, medwords_misinfo = getWordBag(doc_medical_misinfo)
wordFrequency(medwords_misinfo, ntop = 10, normalize = True)

{'diabete': 0.0115,
 'insulin': 0.0075,
 'patient': 0.0072,
 'I': 0.007,
 'people': 0.0066,
 'increase': 0.0066,
 'day': 0.0049,
 'sugar': 0.0049,
 'risk': 0.0046,
 'cell': 0.0045}

# Topic Modelling

In [52]:
# Building a bag of subtitles for each video
# See https://radimrehurek.com/gensim/corpora/dictionary.html
# See https://towardsdatascience.com/end-to-end-topic-modeling-in-python-latent-dirichlet-allocation-lda-35ce4ed6b3e0
# Create a list of words for each video subtitle.
"""
temp = df600[df600["info"] == 1]["subtitle"]
subtitle_lst = [cleanText(subtitle, stopword_lst, return_string = False) for subtitle in temp]
"""

subtitle_lst = cor_allwords_lst
# Convert each word into dictionary. This allows us to perform mapping in the future.
id2word = corpora.Dictionary(subtitle_lst)
# Build a corpus in term document frequency
corpus = [id2word.doc2bow(subtitle) for subtitle in subtitle_lst]

In [53]:
# number of topics
num_topics = 3
# Build LDA model
lda_model = LdaMulticore(corpus=corpus,id2word=id2word, num_topics=num_topics)

# Print the Keyword in the topics
# Possible interpretation: Different aspects of diabetes.
# Word counts on two classes
for i in range(len(lda_model.print_topics())):
    print(lda_model.print_topics()[i])

(0, '0.016*"blood" + 0.015*"insulin" + 0.014*"diabete" + 0.010*"sugar" + 0.010*"go" + 0.010*"glucose" + 0.008*"know" + 0.008*"type" + 0.007*"cell" + 0.007*"level"')
(1, '0.014*"diabete" + 0.013*"blood" + 0.010*"cell" + 0.010*"type" + 0.009*"glucose" + 0.009*"sugar" + 0.007*"insulin" + 0.006*"level" + 0.005*"high" + 0.005*"like"')
(2, '0.022*"blood" + 0.022*"insulin" + 0.021*"diabete" + 0.015*"sugar" + 0.011*"type" + 0.010*"glucose" + 0.007*"level" + 0.006*"cell" + 0.006*"like" + 0.006*"body"')


In [60]:
count = 0
for ent in doc_medical_misinfo.ents:
    count += 1
print(count)

29042


In [61]:
count = 0
for ent in doc_medical_cor.ents:
    count += 1
print(count)

12708


In [80]:
allwords, medwords = getWordBag(doc_medical_misinfo)

In [82]:
mis_medfreqlst

[{'lab test': 0.1429,
  'persistent diarrhea': 0.0952,
  'cause': 0.0952,
  'doctor': 0.0952,
  'Prometheus': 0.0952,
  'diagnostic test': 0.0952,
  'trap': 0.0476,
  'potential': 0.0476,
  'mystery': 0.0476,
  'trip': 0.0476,
  'treatment plan': 0.0476,
  'haunt': 0.0476,
  'I': 0.0476,
  'your doctor': 0.0476},
 {'breakfast': 0.04,
  'leftover juice pancake': 0.04,
  'syrup fresh toast waffle': 0.04,
  'Hashbrown deep-fried': 0.04,
  'Oreo': 0.04,
  'birthday': 0.04,
  'cake': 0.04,
  'Goddamn Krispy Kreme donut': 0.04,
  'ice cream': 0.04,
  'booker t chicken': 0.04,
  'Coke': 0.04,
  'pass': 0.04,
  'potato chip': 0.04,
  'pizza': 0.04,
  'Cuba': 0.04},
 {'heart disease': 0.0811,
  'NIH': 0.0541,
  'risk': 0.0541,
  'type 2 diabete': 0.0541,
  'heart month': 0.027,
  "Valentine's Day": 0.027,
  'National Institute of diabete digestive': 0.027,
  'kidney disease': 0.027,
  'people': 0.027,
  'die': 0.027,
  'year': 0.027,
  'heart': 0.027,
  'learn': 0.027,
  'increase': 0.027,
  'c